In [2]:
pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 1.2/376.0 MB 25.3 MB/s eta 0:00:15
   ---------------------------------------- 3.3/376.0 MB 34.6 MB/s eta 0:00:11
    --------------------------------------- 5.3/376.0 MB 37.5 MB/s eta 0:00:10
    --------------------------------------- 7.8/376.0 MB 41.3 MB/s eta 0:00:09
   - -------------------------------------- 9.9/376.0 MB 42.1 MB/s eta 0:00:09
   - -------------------------------------- 12.1/376.0 MB 50.4 MB/s eta 0:00:08
   - -------------------------------------- 14.7/376.0 MB 50.1 MB/s eta 0:00:08
   - -------------------------------------- 16.9/376.0 MB 50.4 MB/s eta 0:00:08
   -- ------------------------------------- 19.5/376.0 MB 50.4 MB/s eta 0:00:08
   -- ------------------------------------- 22.2/376.0 MB 50.4 MB/s eta 0:00:08
   -- ------------------------------------- 24.7/376.0 MB 54.7 MB/s eta 0:00:07
   -- ------------------------------------- 27.2/376.0

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model

# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
model = Model(inputs=base_model.input, outputs=base_model.output)

# Paths
base_path = os.path.join('..', 'Data', 'train')
output_path = os.path.join('features', 'features_train.csv')


categories = {
    'with_label/clean': 'clean',
    'with_label/dirty': 'dirty',
    'no_label': 'unknown'
}

# Helper function to load and preprocess image
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Image not loaded correctly.")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)  # Preprocess for MobileNetV2
        return image
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# Loop through folders
features = []
for rel_path, label in categories.items():
    full_dir = os.path.join(base_path, rel_path)
    for fname in os.listdir(full_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(full_dir, fname)
            image = preprocess_image(img_path)
            if image is not None:
                image = np.expand_dims(image, axis=0)  # Add batch dimension
                feature_vector = model.predict(image, verbose=0).flatten()  # shape: (1280,)
                features.append({
                    'file': fname,
                    'label': label,
                    **{f'feat_{i}': float(v) for i, v in enumerate(feature_vector)}
                })

# Save to CSV
df = pd.DataFrame(features)
df.to_csv(output_path, index=False)
print(f"Saved {len(df)} feature rows to {output_path}")

Saved 600 feature rows to features\features_train.csv
